In [2]:
! pip install pandas


In [4]:
import random
import pandas as pd
from IPython.display import display
import numpy as np

# Step 1: Define a mapping for skills to numeric codes
skills_map = {
    'Skill_1': 1,
    'Skill_2': 2,
    'Skill_3': 3,
    'Skill_4': 4,
    'Skill_5': 5
}

# Step 2: Define Teams with multiple skills and resource count for each skill
teams = {
    'Team_A': {'resources': 10, 'skills': {skills_map['Skill_1']: 4, skills_map['Skill_2']: 3, skills_map['Skill_3']: 3}},
    'Team_B': {'resources': 8,  'skills': {skills_map['Skill_2']: 2, skills_map['Skill_3']: 3, skills_map['Skill_4']: 3}},
    'Team_C': {'resources': 12, 'skills': {skills_map['Skill_3']: 4, skills_map['Skill_4']: 4, skills_map['Skill_5']: 4}},
    'Team_D': {'resources': 7,  'skills': {skills_map['Skill_1']: 2, skills_map['Skill_4']: 3}},
    'Team_E': {'resources': 5,  'skills': {skills_map['Skill_2']: 2, skills_map['Skill_5']: 3}},
    'Team_F': {'resources': 10, 'skills': {skills_map['Skill_4']: 4, skills_map['Skill_5']: 4}},
    'Team_G': {'resources': 8,  'skills': {skills_map['Skill_1']: 3, skills_map['Skill_3']: 3}},
    'Team_H': {'resources': 6,  'skills': {skills_map['Skill_2']: 3, skills_map['Skill_4']: 3}},
    'Team_I': {'resources': 5,  'skills': {skills_map['Skill_1']: 2, skills_map['Skill_5']: 3}},
    'Team_J': {'resources': 9,  'skills': {skills_map['Skill_3']: 4, skills_map['Skill_5']: 4}}
}

# Step 3: Define 30 Projects with Costs, Benefits, and Numeric Skill Codes
projects = [
    {
        'id': i,
        'cost': random.randint(50000, 300000),
        'benefit': random.randint(100000, 700000),
        'requirements': {
            random.choice(list(teams.keys())): random.randint(2, 6),
            random.choice(list(teams.keys())): random.randint(1, 4)
        },
        'skills_required': {
            random.choice(list(skills_map.values())): random.randint(1, 2)
        },  # Use numeric codes for skills with required count
        'strategic_priority': random.randint(1, 4),
        'regulatory_priority': random.randint(1, 4),
        'vendor_dependency': random.randint(1, 4),
        'dependencies': random.sample(range(i), k=random.randint(0, i))  # Random dependencies on previous projects
    }
    for i in range(30)
]

# Step 4: Initialize Population
def initialize_population(pop_size, num_projects):
    population = []
    for _ in range(pop_size):
        project_selection = {'selection': [random.randint(0, 1) for _ in range(num_projects)]}
        population.append(project_selection)
    return population

# Step 5: Fitness Function with Conflict Tracking using Numeric Skills and Skill Count
def fitness_with_conflicts(project_selection, projects, teams):
    selected_projects = [projects[i] for i, selected in enumerate(project_selection['selection']) if selected]
    team_resource_usage = {team: 0 for team in teams}
    
    # Create a copy of the teams to avoid mutation
    teams_copy = {team: {'resources': team_info['resources'], 'skills': team_info['skills'].copy()} for team, team_info in teams.items()}

    total_benefit = 0
    total_cost = 0
    priority_score = 0
    penalty = 0
    completed_projects = set()

    resource_conflicts = []
    priority_conflicts = []

    for project in selected_projects:
        # Check dependencies
        if not all(dep in completed_projects for dep in project['dependencies']):
            penalty += 50  # Penalty for unmet dependencies
            continue

        # Check team skills and resource availability
        conflict = False
        for team, req in project['requirements'].items():
            if team_resource_usage[team] + req > teams_copy[team]['resources']:
                resource_conflicts.append(f"Resource conflict: Team {team} is over-allocated by {team_resource_usage[team] + req - teams_copy[team]['resources']} resources.")
                penalty += 100  # Penalty for exceeding team resources
                conflict = True
                break

            # Compare numeric skill codes and count of skills
            for required_skill, skill_count in project['skills_required'].items():
                if teams_copy[team]['skills'].get(required_skill, 0) < skill_count:
                    penalty += 50  # Penalty for missing required skill or not enough count
                    conflict = True
                    break
                else:
                    # Deduct the skill count for the team if selected
                    teams_copy[team]['skills'][required_skill] -= skill_count

        # Check for priority conflicts between projects assigned to the same team
        if not conflict:
            for team in project['requirements']:
                if any(p for p in selected_projects if p['id'] != project['id'] and p['requirements'].get(team) and p['strategic_priority'] == project['strategic_priority']):
                    priority_conflicts.append(f"Priority conflict: Team {team} is handling multiple projects with the same priority ({project['strategic_priority']}).")
                    penalty += 30  # Additional penalty for priority conflicts

        if conflict:
            continue  # Skip to next project due to conflict

        # Allocate resources
        for team, req in project['requirements'].items():
            team_resource_usage[team] += req

        # Accumulate the project's benefit, cost, and priority score
        total_benefit += project['benefit']
        total_cost += project['cost']
        priority_score += project['strategic_priority'] * 2 + project['regulatory_priority'] * 3  # Weighted priorities
        penalty += project['vendor_dependency'] * 10  # Slight penalty for higher vendor dependency

        # Mark the project as completed
        completed_projects.add(project['id'])

    # Final fitness score calculation (total benefit - cost + weighted priorities - penalties)
    fitness_score = (total_benefit - total_cost) + priority_score - penalty

    return fitness_score, resource_conflicts, priority_conflicts

# Step 6: Display Initial Problem (All Projects)
def display_initial_projects():
    # Create DataFrame for initial projects display
    data = [{
        'Project ID': project['id'],
        'Cost': project['cost'],
        'Benefits': project['benefit'],
        'Teams Involved': ', '.join(list(project['requirements'].keys())),
        'Dependencies': ', '.join(map(str, project['dependencies'])) if project['dependencies'] else 'None',
        'Strategic Priority': project['strategic_priority'],
        'Regulatory Priority': project['regulatory_priority'],
        'Vendor Dependency': project['vendor_dependency']
    } for project in projects]

    df = pd.DataFrame(data)

    # Display the initial project setup
    print("\nInitial Projects:\n")
    display(df)

# Step 7: Display Ranked Projects with Color (Green for Selected, Red for Non-Selected)
def display_ranked_projects_with_color(project_selection, projects):
    data = []

    for i, selected in enumerate(project_selection['selection']):
        project = projects[i]
        row = {
            'Project ID': project['id'],
            'Selected': selected,
            'Cost': project['cost'],
            'Benefits': project['benefit'],
            'Teams Involved': ', '.join(list(project['requirements'].keys())),
            'Dependencies': ', '.join(map(str, project['dependencies'])) if project['dependencies'] else 'None',
            'Strategic Priority': project['strategic_priority'],
            'Regulatory Priority': project['regulatory_priority'],
            'Vendor Dependency': project['vendor_dependency']
        }
        data.append(row)

    df = pd.DataFrame(data)

    # Style the DataFrame with color based on selection
    def highlight_selection(row):
        color = 'green' if row['Selected'] == 1 else 'red'
        return ['background-color: {}'.format(color) for _ in row]

    styled_df = df.style.apply(highlight_selection, axis=1)
    
    print("\nRanked Projects with Selection:\n")
    display(styled_df)

# Step 8: Genetic Algorithm with Conflict Tracking (returns the best project selection)
def genetic_algorithm_with_conflicts():
    population = initialize_population(20, len(projects))

    for gen in range(50):
        new_population = []
        # Elitism: Keep the best project selections
        population.sort(key=lambda sel: fitness_with_conflicts(sel, projects, teams)[0], reverse=True)
        elites = population[:2]
        new_population.extend(elites)

        # Generate new project selections
        while len(new_population) < 20:
            parent1 = selection(population, projects, teams)
            parent2 = selection(population, projects, teams)
            offspring = crossover(parent1, parent2)
            for child in offspring:
                child = mutate(child, 0.1)
                new_population.append(child)
                if len(new_population) >= 20:
                    break

        population = new_population

    # Return the best project selection
    best_project_selection = max(population, key=lambda sel: fitness_with_conflicts(sel, projects, teams)[0])
    return best_project_selection

# Selection, Crossover, and Mutation functions (unchanged from previous version)
def selection(population, projects, teams):
    tournament_size = 3
    selected = random.sample(population, tournament_size)
    selected_fitness = [fitness_with_conflicts(sel, projects, teams)[0] for sel in selected]
    return selected[np.argmax(selected_fitness)]

def crossover(parent1, parent2):
    point = random.randint(1, len(parent1['selection']) - 1)
    child1_selection = parent1['selection'][:point] + parent2['selection'][point:]
    child2_selection = parent2['selection'][:point] + parent1['selection'][point:]
    return [{'selection': child1_selection}, {'selection': child2_selection}]

def mutate(project_selection, mutation_rate):
    for i in range(len(project_selection['selection'])):
        if random.uniform(0, 1) < mutation_rate:
            project_selection['selection'][i] = 1 - project_selection['selection'][i]
    return project_selection

# Step 9: Invoke the algorithm and display the results
def main():
    # Step 1: Display initial projects
    display_initial_projects()

    # Step 2: Run genetic algorithm to get the best project selection
    best_selection = genetic_algorithm_with_conflicts()

    # Step 3: Display the ranked projects with selection highlighted
    display_ranked_projects_with_color(best_selection, projects)

# Run the main function
main()



Initial Projects:



,Project ID,Cost,Benefits,Teams Involved,Dependencies,Strategic Priority,Regulatory Priority,Vendor Dependency
0,0,257773,413098,"Team_J, Team_A",None,1,3,2
1,1,114038,364515,"Team_E, Team_A",0,2,2,3
2,2,63556,392608,"Team_J, Team_I","0, 1",1,4,3
3,3,203078,317006,"Team_J, Team_B",None,4,1,1
4,4,266968,609716,"Team_B, Team_G",None,2,2,2
5,5,282354,546782,"Team_C, Team_D",None,3,3,2
6,6,220584,545174,"Team_H, Team_J",1,1,3,2
7,7,289729,535250,"Team_E, Team_G","4, 0, 6, 5, 1, 2",3,3,2
8,8,266527,241083,"Team_E, Team_A","3, 5, 1, 2",4,3,4
9,9,157530,650503,"Team_E, Team_B","7, 0",4,2,1



Ranked Projects with Selection:



,Project ID,Selected,Cost,Benefits,Teams Involved,Dependencies,Strategic Priority,Regulatory Priority,Vendor Dependency
0,0,0,257773,413098,"Team_J, Team_A",None,1,3,2
1,1,0,114038,364515,"Team_E, Team_A",0,2,2,3
2,2,0,63556,392608,"Team_J, Team_I","0, 1",1,4,3
3,3,0,203078,317006,"Team_J, Team_B",None,4,1,1
4,4,1,266968,609716,"Team_B, Team_G",None,2,2,2
5,5,0,282354,546782,"Team_C, Team_D",None,3,3,2
6,6,1,220584,545174,"Team_H, Team_J",1,1,3,2
7,7,0,289729,535250,"Team_E, Team_G","4, 0, 6, 5, 1, 2",3,3,2
8,8,0,266527,241083,"Team_E, Team_A","3, 5, 1, 2",4,3,4
9,9,0,157530,650503,"Team_E, Team_B","7, 0",4,2,1
